In [1]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)
 
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np

import proyeccion_rdr.features as features
import proyeccion_rdr.produccion.a05_unidades_de_apoyo as ua
from proyeccion_rdr.produccion.a04_ambulatorio import (
    leer_consultas_medicas
)

from itertools import islice

pd.set_option("display.max_columns", None)

ANIOS_POBLACION = [str(i) for i in range(2017, 2036)]

2025-06-10 17:55:11.418 | INFO     | proyeccion_rdr.config:<module>:11 - PROJ_ROOT path is: C:\Users\javie\OneDrive\Documents\Trabajo\SSMN\2.0-proyeccion-rdr


## Leer insumos de Apoyo Diagnostico

In [5]:
# Lee la proyeccion de consultas medicas
RUTA_CONSULTAS = "../data/interim/3.0_estimacion_boxes_medicos_RDR.xlsx"
consultas_medicas_proyectadas = features.leer_proyeccion_consultas_medicas_especialidad(
    RUTA_CONSULTAS, ANIOS_POBLACION
)

# Lee la cantidad de DCO proyectados
RUTA_DCO = "../data/interim/1.0_estimacion_atencion_cerrada_RDR.xlsx"
dco_proyectados = features.leer_dco_proyectados(RUTA_DCO, ANIOS_POBLACION)

# Lee la cantidad de consultas de urgencias
RUTA_URGENCIAS = "../data/interim/4.0_estimacion_consultas_urgencia.xlsx"
consultas_c1_c2_c3, consultas_c4_c5, consultas_urgencia_proyectadas = (
    features.leer_consultas_urgencia_proyectadas(RUTA_URGENCIAS)
)

In [6]:
# Define las trazadoras a omitir
TRAZADORAS_A_OMITIR = []

# Lee los casos de todos los macroprocesos
RUTA_ARCHIVOS = "../data/interim/0.1_casos_teoricos_diagnosticos.xlsx"
_, casos_macroproceso = features.leer_casos_macroprocesos(RUTA_ARCHIVOS)
casos_macroproceso = casos_macroproceso.query("Diagnostico not in @TRAZADORAS_A_OMITIR")

# Obtiene los casos de area de influencia y hosptializados
casos_area_de_influencia = casos_macroproceso.query("tipo_paciente == 'area_de_influencia'")
casos_hospitalizados_proyectados = casos_macroproceso.query("tipo_paciente == 'hospitalizados'")

# Aisla solamente los casos de cada macroproceso
casos_area_de_influencia = casos_area_de_influencia[ANIOS_POBLACION]
casos_hospitalizados_proyectados = casos_hospitalizados_proyectados[ANIOS_POBLACION]

# Obtiene los casos netamente de atencion abierta
casos_atencion_abierta = casos_area_de_influencia.sub(
    casos_hospitalizados_proyectados, fill_value=0
)
casos_atencion_abierta = casos_atencion_abierta.query("`2017` > 0")

# # Lee los casos de urgencias
casos_urgencia = ua.leer_casos_urgencia(RUTA_URGENCIAS)

# Consolida la cantidad de casos
casos_area_de_influencia = casos_area_de_influencia.sum()
casos_hospitalizados_proyectados = casos_hospitalizados_proyectados.sum()
casos_atencion_abierta = casos_atencion_abierta.sum()

In [7]:
# Define las rutas de los archivos
ruta_raw = "../data/raw/"

# Define las rutas de la base de CAE y GRD
ruta_cae = ruta_raw + "6_ambulatorio/df_procesada_consultas.csv"
ruta_grd = ruta_raw + "7_egresos_grd/df_procesada_grd.csv"
ruta_urgencias = ruta_raw + "8_urgencias/df_procesada_urgencia.csv"

# Define las rutas de los archivos de apoyo diagnostico
ruta_apoyo = ruta_raw + "9_unidades_de_apoyo/"

ruta_laboratorio = ruta_apoyo + "df_procesada_laboratorio_clinico.csv"
ruta_farmacia = ruta_farmacia_amb = ruta_apoyo + "df_procesada_farmacia.csv"
ruta_imagenologia = ruta_apoyo + "df_procesada_imagenologia.csv"
ruta_anatomia = ruta_apoyo + "df_procesada_anatomia_patologica.csv"
ruta_umt = ruta_apoyo + "df_procesada_umt.csv"

In [8]:
# Lee CAE y GRD
consultas_medicas = leer_consultas_medicas(ruta_cae)
df_grd = ua.leer_grd(ruta_grd)
df_urgencias = ua.leer_urgencias(ruta_urgencias)

C:\Users\javie\OneDrive\Documents\Trabajo\SSMN\2.0-proyeccion-rdr\proyeccion_rdr\produccion\a04_ambulatorio.py:90: DtypeWarning: Columns (19,20,21) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(ruta, dtype={"id_paciente": str})
C:\Users\javie\OneDrive\Documents\Trabajo\SSMN\2.0-proyeccion-rdr\proyeccion_rdr\produccion\a05_unidades_de_apoyo.py:21: DtypeWarning: Columns (47,48,49,50,51,52,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,220) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(ruta, dtype={"id_paciente": str})


In [21]:
# Lee las unidades de apoyo
df_laboratorio, df_laboratorio_hosp, df_laboratorio_amb = ua.leer_laboratorio(ruta_laboratorio)
df_farmacia, df_farmacia_hosp, df_farmacia_amb = ua.leer_farmacia(ruta_farmacia)
df_imag, df_imag_hosp, df_imag_amb, df_imag_urg = ua.leer_imagenologia(ruta_imagenologia)
df_anat, df_anat_hosp, df_anat_amb, df_anat_urg = ua.leer_anatomia(ruta_anatomia)

In [ ]:
# Define la agrupacion a calcular
AGRUPACION_EXAMENES = ["ano", "id_paciente"]

# Obtiene los dco totales y consultas por pacientes por anio
consultas_por_pacientes = consultas_medicas.groupby(AGRUPACION_EXAMENES)["fecha_atencion"].count()
dco_por_pacientes = df_grd.groupby(AGRUPACION_EXAMENES)["estancia"].sum()

# Obtiene los examanes por paciente de laboratorio
examenes_ppct_lab_hosp = df_laboratorio_hosp.groupby(AGRUPACION_EXAMENES)["examen"].count()
examenes_ppct_lab_amb = df_laboratorio_amb.groupby(AGRUPACION_EXAMENES)["examen"].count()

# Obtiene las prescripciones por pacientes de farmacia
prescr_ppct_farmacia_hosp = df_farmacia_hosp.groupby(AGRUPACION_EXAMENES)["nombre_articulo"].count()
prescr_ppct_farmacia_amb = df_farmacia_amb.groupby(AGRUPACION_EXAMENES)["nombre_articulo"].count()

In [33]:
# Obtiene los examenes por DCO y consultas para laboratorio
_, examenes_por_dco_lab = ua.calcular_examenes_por_atencion(
    dco_por_pacientes, examenes_ppct_lab_hosp, "estancia", "examen"
)
_, examenes_por_consulta_lab = ua.calcular_examenes_por_atencion(
    consultas_por_pacientes, examenes_ppct_lab_amb, "fecha_atencion", "examen"
)

# Obtiene los examenes por DCO y consultas para laboratorio
_, prescr_por_dco = ua.calcular_examenes_por_atencion(
    dco_por_pacientes, prescr_ppct_farmacia_hosp, "estancia", "nombre_articulo"
)
_, prescr_por_consulta = ua.calcular_examenes_por_atencion(
    consultas_por_pacientes, prescr_ppct_farmacia_amb, "fecha_atencion", "nombre_articulo"
)

## Laboratorio Clinico

En este apartado se quiere saber cuantos examenes de Laboratorio Clinico se haran

In [34]:
print("Laboratorio Clinico - AC - Examenes por DCO")
display(examenes_por_dco_lab)

print("Laboratorio Clinico - AA - Examenes por CME")
display(examenes_por_consulta_lab)

Laboratorio Clinico - AC - Examenes por DCO


,count,id_paciente,mean,std,min,25%,50%,75%,max
ano,,,,,,,,,
2017,6781.0,14336,5.906895,13.123988,0.0,0.000000,2.900000,8.000000,677.000000
2018,7050.0,15511,5.405699,8.059222,0.0,0.000000,2.750000,7.770833,148.000000
2019,7255.0,15704,5.869289,8.475399,0.0,0.000000,3.250000,8.174242,189.800000
2020,4008.0,11966,7.675083,11.139853,0.0,0.636364,4.500000,10.719925,319.000000
2021,4544.0,18269,7.702273,9.850032,0.0,1.000000,4.466667,11.000000,162.000000
2022,5505.0,21862,7.507892,8.870251,0.0,1.285714,4.714286,10.500000,134.571429
2023,6183.0,15579,6.596584,8.709555,0.0,1.000000,4.000000,9.000000,173.777778
2024,6342.0,15844,6.794908,8.613264,0.0,0.878472,4.333333,9.592857,139.000000


Laboratorio Clinico - AA - Examenes por CME


,count,id_paciente,mean,std,min,25%,50%,75%,max
ano,,,,,,,,,
2017,33139.0,8068,0.595854,1.692162,0.0,0.0,0.0,0.000000,52.0
2018,34236.0,8583,0.614689,1.705245,0.0,0.0,0.0,0.000000,24.0
2019,35828.0,8872,0.637850,1.760020,0.0,0.0,0.0,0.000000,27.0
2020,23673.0,6245,0.823679,2.334450,0.0,0.0,0.0,0.000000,29.0
2021,25635.0,8999,0.959706,2.439627,0.0,0.0,0.0,0.500000,37.0
2022,30494.0,9599,0.902342,2.271074,0.0,0.0,0.0,0.500000,37.0
2023,33188.0,9605,0.913539,2.271995,0.0,0.0,0.0,0.500000,32.0
2024,35259.0,9272,0.830948,2.212097,0.0,0.0,0.0,0.142857,30.0


Como se ve, la cantidad de examenes por CME es muy baja, por lo que se ocupara el estandar MINSAL.

In [35]:
# Define constantes
CANTIDAD_EXAMENES_AC_A_HACER = examenes_por_dco_lab["75%"].max()
CANTIDAD_EXAMENES_AA_A_HACER = 3  # Estandar MINSAL
CANTIDAD_EXAMENES_URG_A_HACER = 2.5  # Estandar MINSAL
metricas_para_proyectar_laboratorio = pd.DataFrame(
    {
        "examenes_por_dco": [CANTIDAD_EXAMENES_AC_A_HACER],
        "examenes_por_consulta": [CANTIDAD_EXAMENES_AA_A_HACER],
    }
)

# Obtiene le cantidad de examenes de laboratorio requeridos para atencion cerrada
examenes_laboratorio_ac = dco_proyectados * CANTIDAD_EXAMENES_AC_A_HACER

# Obtiene la cantidad de examenes de laboratorio requeridos para atencion abierta
examanes_laboratorio_aa = consultas_medicas_proyectadas * CANTIDAD_EXAMENES_AA_A_HACER

# Obtiene la cantidad de examenes de laboratorio requeridos para atencion de urgencia
examenes_laboratorio_urgencia = consultas_urgencia_proyectadas * CANTIDAD_EXAMENES_URG_A_HACER

# Suma la cantidad de examenes totales
examenes_laboratorio_proyectados = (
    examanes_laboratorio_aa + examenes_laboratorio_ac + examenes_laboratorio_urgencia
)

In [36]:
# Muestra la produccion actual
produccion_actual_laboratorio = (
    df_laboratorio_amb.groupby("ano").size() + df_laboratorio_hosp.groupby("ano").size()
)

In [37]:
display(produccion_actual_laboratorio)
display(round(examenes_laboratorio_proyectados))

ano
2015    468546
2016    488454
2017    507761
2018    485015
2019    554398
2020    425257
2021    504242
2022    549272
2023    522606
2024    514917
dtype: int64

2017    1615383.0
2018    1656239.0
2019    1698665.0
2020    1740298.0
2021    1764353.0
2022    1779607.0
2023    1793844.0
2024    1808703.0
2025    1821813.0
2026    1830549.0
2027    1834446.0
2028    1834881.0
2029    1833165.0
2030    1831124.0
2031    1829603.0
2032    1826613.0
2033    1821417.0
2034    1813813.0
2035    1807785.0
dtype: float64

In [38]:
produccion_promedio_laboratorio = produccion_actual_laboratorio[
    produccion_actual_laboratorio.index != 2025
].mean()
crecimiento_laboratorio = examenes_laboratorio_proyectados["2035"] / produccion_promedio_laboratorio
print(f"Aumento de Laboratorio: {crecimiento_laboratorio:.2f} veces")

Aumento de Laboratorio: 3.60 veces


## Farmacia

In [39]:
print("Farmacia - AC - Prescripciones por DCO")
display(prescr_por_dco)

print("Farmacia - AA - Prescripciones por CME")
display(prescr_por_consulta)

Farmacia - AC - Prescripciones por DCO


,count,id_paciente,mean,std,min,25%,50%,75%,max
ano,,,,,,,,,
2020,4008.0,4423.0,3.138212,3.272902,0.0,1.00,2.333333,4.200000,103.500000
2021,4544.0,5103.0,3.354054,3.297278,0.0,1.00,2.500000,4.733333,68.666667
2022,5505.0,6302.0,3.415716,3.301487,0.0,1.25,2.625000,4.700000,73.000000
2023,6183.0,7273.0,3.463834,3.344725,0.0,1.50,2.700000,4.666667,91.000000
2024,6342.0,9779.0,3.861155,3.065502,0.0,1.80,3.000000,5.250000,56.000000


Farmacia - AA - Prescripciones por CME


,count,id_paciente,mean,std,min,25%,50%,75%,max
ano,,,,,,,,,
2020,23673.0,8631.0,0.294377,0.915475,0.0,0.0,0.0,0.083333,48.00
2021,25635.0,14119.0,0.346525,0.860269,0.0,0.0,0.0,0.333333,16.00
2022,30494.0,16510.0,0.362413,1.018257,0.0,0.0,0.0,0.333333,63.00
2023,33188.0,17135.0,0.358337,0.962443,0.0,0.0,0.0,0.333333,55.00
2024,35259.0,17869.0,0.372922,0.943090,0.0,0.0,0.0,0.333333,28.25


In [40]:
# Define constantes
PRESCRIPCIONES_POR_DCO = prescr_por_dco["75%"].max()
PRESCRIPCIONES_POR_CME = 3 # Estandar MINSAL
PRESCRIPCIONES_POR_CU_C1_C2_C3 = 3 # Estandar MINSAL
PRESCRIPCIONES_POR_CU_C4_C5 = 1 # Estandar MINSAL 
metricas_para_proyectar_farmacia = pd.DataFrame(
    {
        "prescripciones_por_dco": [PRESCRIPCIONES_POR_DCO],
        "prescripciones_por_consulta": [PRESCRIPCIONES_POR_CME],
    }
)

In [56]:
# # Obtiene la cantidad de prescripciones
prescripciones_hosp = dco_proyectados * PRESCRIPCIONES_POR_DCO
prescripciones_amb = consultas_medicas_proyectadas * PRESCRIPCIONES_POR_CME
prescripciones_c1_c2_c3 = consultas_c1_c2_c3 * PRESCRIPCIONES_POR_CU_C1_C2_C3
prescripciones_c4_c5 = consultas_c4_c5 * PRESCRIPCIONES_POR_CU_C4_C5
prescripciones_urg = prescripciones_c1_c2_c3 + prescripciones_c4_c5

# # Obtiene el total de prescripciones
prescripciones_proyectadas = (
    prescripciones_hosp + prescripciones_amb + prescripciones_urg
)

In [42]:
produccion_actual_farmacia = (
    df_farmacia_amb.groupby("ano").size() + df_farmacia_hosp.groupby("ano").size()
)
display(produccion_actual_farmacia)

ano
2020    159394
2021    202696
2022    221254
2023    235168
2024    248106
dtype: int64

In [43]:
display(round(prescripciones_proyectadas))

2017    1218106.0
2018    1249260.0
2019    1281138.0
2020    1312216.0
2021    1330138.0
2022    1341602.0
2023    1352292.0
2024    1363499.0
2025    1373405.0
2026    1379955.0
2027    1382822.0
2028    1383157.0
2029    1381881.0
2030    1380254.0
2031    1378861.0
2032    1376515.0
2033    1372507.0
2034    1366660.0
2035    1361920.0
dtype: float64

In [44]:
produccion_promedio_farmacia = produccion_actual_farmacia.mean()
crecimiento_farmacia = prescripciones_proyectadas["2035"] / produccion_promedio_farmacia
print(f"Aumento de farmacia: {crecimiento_farmacia:.2f} veces")

Aumento de farmacia: 6.38 veces


## Anatomia Patologica

In [48]:
resultados_anatomia = ua.estimar_unidad_completa(
    df_unidad=df_anat,
    df_unidad_hosp=df_anat_hosp,
    df_unidad_amb=df_anat_amb,
    df_unidad_urg=df_anat_urg, # Notar que es un DataFrame vacio
    df_grd=df_grd,
    df_consultas=consultas_medicas,
    df_urgencias=df_urgencias,
    casos_hospitalizados_proyectados=casos_hospitalizados_proyectados,
    casos_atencion_abierta_proyectados=casos_atencion_abierta,
    casos_urgencia_proyectados=casos_urgencia,
    anios_poblacion=ANIOS_POBLACION,
    estimar_examenes_por_paciente_utilizando_glosa=True,
)

No hay anios que se superpongan
No hay anios que se superpongan
No hay anios que se superpongan
No hay anios que se superpongan
No hay anios que se superpongan
No hay anios que se superpongan
No hay anios que se superpongan
No hay anios que se superpongan
No hay anios que se superpongan
Metricas utilizadas para proyectar


,porcentaje_a_realizar_hosp,porcentaje_a_realizar_amb,porcentaje_a_realizar_urg,examenes_por_paciente_hosp,examenes_por_paciente_amb,examenes_por_paciente_urg,modificaciones_a_metricas
801001.0,1.2%,0.0%,-,3.00,5.50,-,
801002.0,0.5%,0.0%,-,3.25,3.00,-,
801005.0,2.9%,0.4%,-,3.00,5.00,-,
801006.0,0.3%,0.0%,-,6.00,22.50,-,
801007.0,0.7%,0.0%,-,4.00,4.00,-,
801008.0,16.8%,1.9%,-,1.00,3.00,-,
glosa_examen,-,-,-,-,-,0.00,


Proyeccion Unidad


index,2017,2018,2019,2020,2021,2022,2023,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034,2035
AMBULATORIO,5128.220847,5272.726440,5412.043635,5542.488773,5614.326719,5659.623745,5701.196611,5744.264290,5780.895855,5805.204229,5818.755129,5826.641860,5828.671178,5826.324736,5820.520052,5811.071470,5794.808562,5771.220098,5750.309201
HOSPITALIZADO,4512.139033,4630.096867,4754.592492,4878.675760,4952.097824,4997.630044,5036.637506,5073.398673,5103.535731,5124.378884,5135.176851,5138.306846,5134.841201,5128.060904,5124.008231,5117.438879,5106.623863,5089.784904,5074.122632
TOTAL,9640.359880,9902.823307,10166.636127,10421.164533,10566.424543,10657.253789,10737.834117,10817.662963,10884.431586,10929.583113,10953.931981,10964.948706,10963.512380,10954.385641,10944.528283,10928.510350,10901.432426,10861.005001,10824.431833


Producción Actual Unidad


ano
2015    3084
2016    2638
2017    2796
2018    3370
2019    3800
2020    2415
2021    2996
2022    2756
2023    2803
2024    3270
Name: glosa_examen, dtype: int64

Aumento de Unidad: 3.62 veces


## Imagenologia

In [49]:
resultados_imag = ua.estimar_unidad_completa(
    df_unidad=df_imag,
    df_unidad_hosp=df_imag_hosp,
    df_unidad_amb=df_imag_amb,
    df_unidad_urg=df_imag_urg,
    df_grd=df_grd,
    df_consultas=consultas_medicas,
    df_urgencias=df_urgencias,
    casos_hospitalizados_proyectados=casos_hospitalizados_proyectados,
    casos_atencion_abierta_proyectados=casos_atencion_abierta,
    casos_urgencia_proyectados=casos_urgencia,
    anios_poblacion=ANIOS_POBLACION,
    estimar_examenes_por_paciente_utilizando_glosa=True,
)

No hay anios que se superpongan
No hay anios que se superpongan
No hay anios que se superpongan
No hay anios que se superpongan
No hay anios que se superpongan
No hay anios que se superpongan
No hay anios que se superpongan
No hay anios que se superpongan
No hay anios que se superpongan
No hay anios que se superpongan
No hay anios que se superpongan
No hay anios que se superpongan
No hay anios que se superpongan
No hay anios que se superpongan
No hay anios que se superpongan
No hay anios que se superpongan
No hay anios que se superpongan
No hay anios que se superpongan
No hay anios que se superpongan
No hay anios que se superpongan
No hay anios que se superpongan
No hay anios que se superpongan
No hay anios que se superpongan
No hay anios que se superpongan
No hay anios que se superpongan
No hay anios que se superpongan
No hay anios que se superpongan
No hay anios que se superpongan
No hay anios que se superpongan
No hay anios que se superpongan
No hay anios que se superpongan
No hay a

,porcentaje_a_realizar_hosp,porcentaje_a_realizar_amb,porcentaje_a_realizar_urg,examenes_por_paciente_hosp,examenes_por_paciente_amb,examenes_por_paciente_urg,modificaciones_a_metricas
glosa_examen,,,,,,,
ANGIOTAC DE ABDOMEN,0.3%,0.0%,0.0%,1.00,2.00,1.00,
ANGIOTAC DE CEREBRO,0.3%,0.0%,0.0%,1.00,1.00,1.50,
ANGIOTAC DE CUELLO,0.1%,0.0%,0.0%,1.00,1.25,1.00,
ANGIOTAC DE EXTREMIDAD SUPERIOR DERECHA (UNILATERAL),-,0.0%,-,-,1.00,-,
ANGIOTAC DE EXTREMIDAD SUPERIOR IZQUIERDA (UNILATERAL),-,0.0%,-,-,1.00,-,
ANGIOTAC DE EXTREMIDADES INFERIORES (BILATERAL),0.0%,0.0%,0.0%,2.00,1.00,1.00,
ANGIOTAC DE PELVIS,0.1%,0.0%,0.0%,1.25,1.00,1.00,
ANGIOTAC DE TORAX,1.3%,0.1%,0.0%,1.00,1.00,1.50,
APOYO FLUOROSCOPICO A PROCEDIMIENTOS INTRAOPERATORIOS Y/O BIOPSIA (NO INCLUYE EL PROC.),0.0%,0.0%,-,2.00,1.00,-,


Proyeccion Unidad


index,2017,2018,2019,2020,2021,2022,2023,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034,2035
AMBULATORIO,72398.969635,74439.064316,76405.910455,78247.503060,79261.693638,79901.185988,80488.101562,81096.120531,81613.276020,81956.455710,82147.764016,82259.106956,82287.756373,82254.629879,82172.680771,82039.287996,81809.692230,81476.676047,81181.461118
HOSPITALIZADO,44282.091776,45439.728895,46661.527833,47879.279905,48599.843382,49046.696179,49429.515061,49790.288821,50086.053638,50290.608155,50396.579305,50427.297046,50393.285240,50326.743465,50286.970563,50222.499008,50116.360539,49951.103142,49797.393748
URGENCIA,89399.806330,92369.543372,95202.148952,97827.369492,99277.757751,100202.600299,101180.636061,102294.396608,103357.439898,104172.460056,104702.197780,104999.368928,105167.697328,105388.988435,105567.573530,105635.879167,105520.440271,105220.722812,105065.439352
TOTAL,206080.867741,212248.336583,218269.587240,223954.152457,227139.294772,229150.482466,231098.252685,233180.805960,235056.769556,236419.523922,237246.541101,237685.772929,237848.738941,237970.361780,238027.224864,237897.666171,237446.493039,236648.502001,236044.294219


Producción Actual Unidad


ano
2020    42959
2021    59719
2022    75422
2023    79111
2024    79605
Name: glosa_examen, dtype: int64

Aumento de Unidad: 3.50 veces


## Guardar archivos

In [46]:
# Consolida los casos utilizados para proyectar las unidades de apoyo
resumen_casos_apoyo_dco_cme = pd.DataFrame(
    {
        "DCO Proyectados": dco_proyectados,
        "CME Proyectados": consultas_medicas_proyectadas,
    }
).transpose()
resumen_casos_apoyo_dco_cme["utilizado_en"] = "Laboratorio Clinico - Farmacia"

# Une los casos de apoyo
resumen_casos_apoyo = pd.concat(
    [
        resumen_casos_apoyo_dco_cme,
    ]
).reset_index(names="Trazadoras")

In [58]:
a_guardar = {
    "DCO, CME y Casos Proyecciones": {
        "Casos Utilizados para Proyectar": resumen_casos_apoyo,
    },
    "Laboratorio Clinico": {
        "Exámenes por DCO Históricos": examenes_por_dco_lab.reset_index(),
        "Exámenes por CME Históricos": examenes_por_consulta_lab.reset_index(),
        "Métricas seleccionadas para proyectar": metricas_para_proyectar_laboratorio.reset_index(),
        "Exámenes Proyectados - AC ": examenes_laboratorio_ac.reset_index(),
        "Exámenes Proyectados - AA ": examanes_laboratorio_aa.reset_index(),
        "Exámenes Proyectados - URG": examenes_laboratorio_urgencia.reset_index(),
        "Exámenes Proyectados - Totales": examenes_laboratorio_proyectados.reset_index(),
    },
    "Farmacia": {
        "Prescripciones por DCO Históricas": prescr_por_dco.reset_index(),
        "Prescripciones por CME Históricas": prescr_por_consulta.reset_index(),
        "Métricas seleccionadas para proyectar": metricas_para_proyectar_farmacia.reset_index(),
        "Prescripciones Proyectadas - AC": prescripciones_hosp.reset_index(),
        "Prescripciones Proyectadas - AA": prescripciones_amb.reset_index(),
        "Prescripciones Proyectadas - URG": prescripciones_urg.reset_index(),
        "Prescripciones Proyectada - Total": prescripciones_proyectadas.reset_index(),
    },
    "Imagenologia": dict(islice(resultados_imag.items(), 7)),
}

In [59]:
# Guarda archivos de Apoyo
ruta_a_guardar = "../data/interim/5.0_estimacion_unidades_de_apoyo_RDR.xlsx"
ua.guardar_dict_en_excel(ruta_a_guardar, a_guardar, espacio_filas=3)